# Open Clusters Analytics - 02_Membership analysis of an open cluster

In [ ]:
# Import neccessary libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# =============================================================================
# 1. INITIAL PARAMETERS OF THE CLUSTER
# =============================================================================

cluster_name='M37'

custom_simbad = Simbad()

# Reset SIMBAD to basic configuration
Simbad.reset_votable_fields()

# Add specific fields
custom_simbad.add_votable_fields('otype', 'dim', 'plx', 'pmra', 'pmdec','dim')

result = custom_simbad.query_object(cluster_name)

print(f"Number of results: {len(result)}")

# Show basic information from SIMBAD
row = result[0]
ra_center = row['ra']
dec_center = row['dec']
radius = row['galdim_majaxis']/2

print(f"\nBasic information:")
print(f"  Name: {row['main_id']}")
print(f"  Type: {row['otype']}")
print(f"  RA: {row['ra']:.6f}º")
print(f"  Dec: {row['dec']:.6f}º")
print(f"  PmRA: {row['pmra']} (mas/yr)") # proper motion ra mas/yr
print(f"  PmDec: {row['pmdec']} (mas/yr)") # proper motion dec mas/yr
print(f"  galdim_majaxis: {row['galdim_majaxis']} (arcmin)") #major axis arcmin
print(f"  galdim_minaxis: {row['galdim_minaxis']} (arcmin)") #minor axis arcmin
print(f"  Radius: {row['galdim_majaxis']/2} (arcmin)")
print(f"  Parallax: {row['plx_value']} (mas)")       


In [ ]:
# =============================================================================
# 2. FUNCTION TO DOWNLOAD DATA FROM GAIA
# =============================================================================

def download_gaia_data(ra, dec, radius):
    """
    Download data from Gaia DR3 in a circular region around coordinates of the center
    """
    # Convert radio to degrees
    radius_deg = radius / 60.0
    
    # Optimized query
    query = f"""
    SELECT 
        source_id,
        ra, dec,
        pmra, pmdec,
        pmra_error, pmdec_error,
        parallax, parallax_error,
        phot_g_mean_mag,
        phot_bp_mean_mag,
        phot_rp_mean_mag,
        bp_rp,
        radial_velocity,
        radial_velocity_error,
        ruwe,
        astrometric_excess_noise
    FROM gaiadr3.gaia_source
    WHERE CONTAINS(
        POINT('ICRS', ra, dec),
        CIRCLE('ICRS', {ra}, {dec}, {radius_deg})
    ) = 1
    AND parallax > 0
    AND parallax/parallax_error > 5
    AND pmra IS NOT NULL
    AND pmdec IS NOT NULL
    AND phot_g_mean_mag < 20
    AND ruwe < 1.4
    ORDER BY phot_g_mean_mag ASC
    """
    print(f"\nDownloading data from Gaia DR3...")
    print(f"Center: RA={ra:.3f}°, DEC={dec:.3f}°")
    print(f"Radius: {radius} arcmin")
    
    # Request credentials to connect to Gaia
    authenticated = False
    username = "user"
    password = "pass"

    try:
        Gaia.login(user=username, password=password)
        print("Login successful")
        authenticated = True
        
    except Exception as e:
        print(f"Login error: {e}")
        print("continue without login")


    try:
        if authenticated:
            # Launch asynchronous query (better for big query)
            print('Launching async job')
            job = Gaia.launch_job_async(query)

            # Esperar a que termine
            import time
            while not job.is_finished():
                print(f"Job status: {job.get_phase()}")
                if job.is_error():
                    print(f"Job failed: {job.get_error()}")
                    return None
                time.sleep(5)
            
            print("Job completed, retrieving results...")
            results = job.get_results()
            
            print(f"Downloaded {len(results)} stars async")

        else:
            # Launch synchronous query (anonymous)
            print('Launching sync job')        
            job = Gaia.launch_job(query)
            results = job.get_results()
            print(f"Downloaded {len(results)} stars sync")
            
        print(f"Downloaded {len(results)} stars")
        
    except Exception as e:
        print(f"Error executing the query in Gaia: {e}")

    # Convert to a Pandas DataFrame
    df = results.to_pandas()
    
    # Clean rows with mandatory data for membership analysis
    df = df.dropna(subset=['parallax', 'pmra', 'pmdec', 'phot_g_mean_mag'])
    
    # Calculate BP-RP
    df['bp_rp'] = df['phot_bp_mean_mag'] - df['phot_rp_mean_mag']

    print(f"After cleaning: {len(df)} stars")
    
    return df



In [ ]:

# Download data
try:
    radius= radius + 1 #increase +1 arcmin the radius
    gaia_data = download_gaia_data(ra_center, dec_center, radius)
    print(f"\n Data succesfully downloaded: {len(gaia_data)} stars")
except Exception as e:
    print(f"Error downloading data: {e}")


In [ ]:
gaia_data.head(5)

In [ ]:
# =============================================================================
# 3. DATA VISUALIZATION 
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Celestial map
axes[0,0].scatter(gaia_data['ra'], gaia_data['dec'], alpha=0.6, s=10)
axes[0,0].set_xlabel('RA (º)')
axes[0,0].set_ylabel('DEC (º)')
axes[0,0].set_title(f'Spatial distribution - {cluster_name}')
axes[0,0].grid(True, alpha=0.3)

# Proper motions diagram
axes[0,1].scatter(gaia_data['pmra'], gaia_data['pmdec'], alpha=0.6, s=10)
axes[0,1].set_xlabel('pmRA (mas/year)')
axes[0,1].set_ylabel('pmDEC (mas/year)')
axes[0,1].set_title('Proper motions')
axes[0,1].grid(True, alpha=0.3)

# Parallax histogram
axes[1,0].hist(gaia_data['parallax'], bins=50, alpha=0.7, edgecolor='black')
axes[1,0].set_xlabel('Parallax (mas)')
axes[1,0].set_ylabel('Number of stars')
axes[1,0].set_title('Parallax distribution')
axes[1,0].grid(True, alpha=0.3)

# Colour-magnitude diagram
axes[1,1].scatter(gaia_data['bp_rp'], gaia_data['phot_g_mean_mag'], 
                 alpha=0.6, s=10)
axes[1,1].set_xlabel('BP-RP (mag)')
axes[1,1].set_ylabel('G (mag)')
axes[1,1].set_title('Colour-Magnitude diagram')
axes[1,1].invert_yaxis()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('basic_diagrams.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Function to convert BP-RP to RGB colour
def bp_rp_to_rgb(bp_rp):
    """
    Convert BP-RP to RGB colour (aprox)
    Based on the empiric relationship between photometric colour and temperature
    """
    # Normalize BP-RP to tipical range (0 a 2.5)
    bp_rp_norm = np.clip(bp_rp, -0.5, 3.0)
    
    # Mapeo aproximado de BP-RP a colores estelares
    # BP-RP < 0: blue (hot stars)
    # BP-RP ~ 0.5: white (A type)
    # BP-RP ~ 1.0: yellow (G type, like our Sun)
    # BP-RP > 1.5: red (cold stars)
    
    colors = []
    for color in bp_rp_norm:
        if color < 0.0:  # Blue(estrellas O, B)
            r, g, b = 0.6, 0.8, 1.0
        elif color < 0.3:  # Blue-White (B, A )
            r, g, b = 0.8, 0.9, 1.0
        elif color < 0.6:  # White (A, F)
            r, g, b = 1.0, 1.0, 0.95
        elif color < 1.0:  # Yellow-White (G)
            r, g, b = 1.0, 1.0, 0.8
        elif color < 1.4:  # Yellow-Orange (K)
            r, g, b = 1.0, 0.85, 0.6
        elif color < 2.0:  # Orange (K, M )
            r, g, b = 1.0, 0.65, 0.4
        else:  # Red (M)
            r, g, b = 1.0, 0.4, 0.2
            
        colors.append((r, g, b))
    
    return colors



In [ ]:
# =============================================================================
# 3. DATA VISUALIZATION WITH COLOURS
# =============================================================================

# Calcular colores para cada estrella
stellar_colors = bp_rp_to_rgb(gaia_data['bp_rp'])

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Celestial map
axes[0,0].scatter(gaia_data['ra'], gaia_data['dec'], c=stellar_colors, alpha=0.6, s=10, edgecolors='none')
axes[0,0].set_xlabel('RA (º)')
axes[0,0].set_ylabel('DEC (º)')
axes[0,0].set_title(f'Spatial distribution - {cluster_name}')
axes[0,0].grid(True, alpha=0.3)

# Proper motions diagram
axes[0,1].scatter(gaia_data['pmra'], gaia_data['pmdec'],  c=stellar_colors, alpha=0.6, s=10, edgecolors='none')
axes[0,1].set_xlabel('pmRA (mas/year)')
axes[0,1].set_ylabel('pmDEC (mas/year)')
axes[0,1].set_title('Proper motions')
axes[0,1].grid(True, alpha=0.3)

# Parallax histogram
axes[1,0].hist(gaia_data['parallax'], bins=50, alpha=0.7, edgecolor='black')
axes[1,0].set_xlabel('Parallax (mas)')
axes[1,0].set_ylabel('Number of stars')
axes[1,0].set_title('Parallas distribution')
axes[1,0].grid(True, alpha=0.3)

# Colour-magnitude diagram
axes[1,1].scatter(gaia_data['bp_rp'], gaia_data['phot_g_mean_mag'], c=stellar_colors,
                 alpha=0.6, s=10)
axes[1,1].set_xlabel('BP-RP (mag)')
axes[1,1].set_ylabel('G (mag)')
axes[1,1].set_title('Colour-Magnitude diagram')
axes[1,1].invert_yaxis()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('basic_diagrams_color.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 4. MEMBERSHIP ANALYSIS USING CLUSTERING ALGORITHMS
# =============================================================================

def cluster_membership_analysis(df, method='dbscan'):
    """
    Membership analysis using clustering algorithms with proper motions and parallax
    """
    print(f"\n=== MEMBERSHIP ANALYSIS ===")
    
    # Prepare data for clustering (pmra, pmdec, parallax)
    features = ['pmra', 'pmdec', 'parallax']
    X = df[features].copy()
    
    # Scale data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    if method == 'dbscan':
        # DBSCAN clustering
        # Standard parameters (To be reviewed in future posts)
        clustering = DBSCAN(eps=0.3, min_samples=10)
        cluster_labels = clustering.fit_predict(X_scaled)
        
        print(f"DBSCAN found {len(np.unique(cluster_labels))-1} clusters")
        print(f"Stars classified as noise: {np.sum(cluster_labels == -1)}")
    
    # Add labels to DataFrame
    df['cluster_label'] = cluster_labels
    
    # Identify the cluster 
    if len(np.unique(cluster_labels[cluster_labels != -1])) > 0:
        cluster_counts = pd.Series(cluster_labels[cluster_labels != -1]).value_counts()
        main_cluster_label = cluster_counts.index[0]
        
        # Add tag to the members
        df['is_member'] = (df['cluster_label'] == main_cluster_label)
        
        n_members = df['is_member'].sum()
        print(f"Principal cluster (label {main_cluster_label}): {n_members} stars")
        
    else:
        df['is_member'] = False
        print("Significative clusters not found")
    
    return df

# Launch membership analysis
gaia_data = cluster_membership_analysis(gaia_data)

In [ ]:
gaia_data

In [ ]:
# =============================================================================
# 5. RESULT VISUALIZATION
# =============================================================================

# Create graph
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Different colors for members and non-members
colors = ['dodgerblue' if member else 'lightblue' for member in gaia_data['is_member']]
sizes = [20 if member else 5 for member in gaia_data['is_member']]

# 1. Spatial distribution with membership
axes[0,0].scatter(gaia_data['ra'], gaia_data['dec'], c=colors, s=sizes, alpha=0.7)
axes[0,0].set_xlabel('RA (º)')
axes[0,0].set_ylabel('DEC (º)')
axes[0,0].set_title('Spatial distribution\n(Blue: members of the cluster)')
axes[0,0].grid(True, alpha=0.3)

# 2. Proper motions with membership
axes[0,1].scatter(gaia_data['pmra'], gaia_data['pmdec'], c=colors, s=sizes, alpha=0.7)
axes[0,1].set_xlabel('pmRA (mas/year)')
axes[0,1].set_ylabel('pmDEC (mas/year)')
axes[0,1].set_title('Proper motions\n(Blue: members of the cluster)')
axes[0,1].grid(True, alpha=0.3)

# 3. Parallax with membership
members = gaia_data[gaia_data['is_member']]
non_members = gaia_data[~gaia_data['is_member']]

axes[0,2].hist(non_members['parallax'], bins=30, alpha=0.5, 
              color='lightblue', label='Field', density=True)
axes[0,2].hist(members['parallax'], bins=15, alpha=0.8, 
              color='dodgerblue', label='Cluster', density=True)
axes[0,2].set_xlabel('Parallax (mas)')
axes[0,2].set_ylabel('Density')
axes[0,2].set_title('Parallax distribution')
axes[0,2].legend()
axes[0,2].grid(True, alpha=0.3)

# 4. Colour-Magnitude Diagram
axes[1,0].scatter(non_members['bp_rp'], non_members['phot_g_mean_mag'], 
                 c='lightblue', s=5, alpha=0.5, label='Field')
axes[1,0].scatter(members['bp_rp'], members['phot_g_mean_mag'], 
                 c='dodgerblue', s=20, alpha=0.8, label='Cluster')
axes[1,0].set_xlabel('BP-RP (mag)')
axes[1,0].set_ylabel('G (mag)')
axes[1,0].set_title('Colour-Magnitude Diagram')
axes[1,0].invert_yaxis()
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 5. Magnitudes distribution
axes[1,1].hist(non_members['phot_g_mean_mag'], bins=30, alpha=0.5, 
              color='lightblue', label='Field', density=True)
axes[1,1].hist(members['phot_g_mean_mag'], bins=15, alpha=0.8, 
              color='dodgerblue', label='Cluster', density=True)
axes[1,1].set_xlabel('G (mag)')
axes[1,1].set_ylabel('Density')
axes[1,1].set_title('Función de luminosidad')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

# 6. Cluster statistics
axes[1,2].axis('off')
if len(members) > 0:
    stats_text = f"""CLUSTER STATISTICS

Name: {cluster_name}

Total number of stars: {len(gaia_data)}
Cluster members: {len(members)}
Percentage of members: {len(members)/len(gaia_data)*100:.1f}%

CLUSTER PROPERTIES:
mean pmRA: {members['pmra'].mean():.2f} ± {members['pmra'].std():.2f} mas/year
mean pmDEC: {members['pmdec'].mean():.2f} ± {members['pmdec'].std():.2f} mas/year
mean Parallax: {members['parallax'].mean():.2f} ± {members['parallax'].std():.2f} mas
Distance: ~{1000/members['parallax'].mean():.0f} pc

COORDINATES OF THE CENTRE:
RA centre: {members['ra'].mean():.3f}°
DEC centre: {members['dec'].mean():.3f}°
Spatial dispersion: {np.sqrt(members['ra'].std()**2 + members['dec'].std()**2):.3f}°
"""
else:
    stats_text = "No members were identified"

axes[1,2].text(0.05, 0.95, stats_text, transform=axes[1,2].transAxes, 
               verticalalignment='top', fontfamily='monospace', fontsize=10)

plt.tight_layout()
plt.savefig('basic_diagrams_membership.png', dpi=300, bbox_inches='tight')
plt.show()